[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeyabbalas/py-icare/blob/master/demo/Absolute%20risk%20over%20split%20intervals.ipynb)

# Absolute risk over split intervals

In this example, we demonstrate the use of iCARE to build and apply absolute risk models over split intervals. This option allows the user to build absolute risk models that relax the proportional hazards assumption, to some extent, by allowing the relationship between risk factors and the outcome to vary over time. For example, it is well-documented that the relationships between certain risk factors, such as body mass index, and breast cancer are different among pre-menopausal and post-menopausal women. Using `compute_absolute_risk_split_interval()`, users can specify a different set of relative risks, before and after a cut-point of 50 years (the median age of menopause), through the parameters `model_log_relative_risk_before_cutpoint_path` and `model_log_relative_risk_after_cutpoint_path`. The cutpoint is the age at which the relative risks change in the population. This function is also useful when the distribution of risk factors varies with age.

In [ ]:
! pip install pyicare --quiet

In [3]:
import pathlib

import pandas as pd
import requests

import icare

seed = 1234

To specify a covariate model in iCARE, we need to provide: 1) a text file containing the covariate model formula description using the [Patsy formula language](https://patsy.readthedocs.io/en/latest/formulas.html) (`model_covariate_formula_before_cutpoint_path`), if the model formula changes after the cut-point, you must also provide the other formula at `model_covariate_formula_after_cutpoint_path`; 2) the breast cancer log odds ratios associated with each risk factor in the covariate model (`model_log_relative_risk_before_cutpoint_path`), if the relative risks change after the cut-point, provide them at `model_log_relative_risk_after_cutpoint_path`; 3) a reference dataset describing the distribution of the classical risk factors in the underlying population (`model_reference_dataset_before_cutpoint_path`) if the risk factor distributions change after the cut-point, provide them at `model_reference_dataset_after_cutpoint_path`; 4) a set of profiles, specifying the classical risk factors of individuals for whom, the absolute risk is to be estimated (`apply_covariate_profile_before_cutpoint_path`), if the risk factors for the same individuals are measured after the cut-point, provide them at `apply_covariate_profile_after_cutpoint_path`, 5) the marginal age-specific incidence rates of breast cancer (`model_disease_incidence_rates_path`), and 6) optionally, the age-specific incidence rates of competing risks (`model_competing_incidence_rates_path`). We include them in this example.

To specify a SNP model using the special option, we must additionally input files containing: 1) the SNP information (`model_snp_info_path`), that has three columns named `snp_name`, `snp_odds_ratio`, and `snp_freq` corresponding to the SNP name, their odds ratios in association with breast cancer risk, and their minor allele frequencies, respectively, and 2) a set of profiles, specifying the SNPs of individuals (same as those specified in the covariate profile) for whom, the absolute risk is to be estimated (`apply_snp_profile_path`). Since the SNPs in Genome-Wide Association Studies (GWAS) measure germline variants, they cannot vary after the cut-point and therefore iCARE does not provide any option to specify different genetic parameters after the cut-point.

The covariate model described in the file specified below is based on a logistic regression model adjusted for cohort and fine categories of age in the Breast and Prostate Cancer Cohort Consortium ([Campa et al. 2011](https://pubmed.ncbi.nlm.nih.gov/21791674/), [Joshi et al. 2014](https://pubmed.ncbi.nlm.nih.gov/25255808/), and [Maas et al. 2016](https://pubmed.ncbi.nlm.nih.gov/27228256/)). The reference dataset was created by simulation from [the National Health Interview Survey (NHIS)](ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NHIS/2010/srvydesc.pdf) and [the National Health and Nutrition Examination Survey (NHANES)](https://wwwn.cdc.gov/nchs/nhanes/default.aspx), which are representative of the US population. `breast_cancer_72_snps_info.csv` contains published information on the odds-ratios and allele frequencies of 72 SNPs identified, among a larger set of markers, to be associated with breast cancer risk by [a recent genome-wide association study](https://www.nature.com/articles/nature24284) (Michailidou et al., 2017). `age_specific_breast_cancer_incidence_rates.csv` contains age-specific incidence rates of breast cancer from [Surveillance, Epidemiology and End Results (SEER) Program](https://seer.cancer.gov/), and `age_specific_all_cause_mortality_rates.csv` has age-specific incidence rates of all-cause mortality from [the CDC WONDER database](https://wonder.cdc.gov/). We indicate `model_family_history_variable_name = "family_history"` to allow the software to properly attenuate the log odds ratio for family history to account for the addition of the 72 SNPs.

In [4]:
# Data files URLs
github_source = "https://raw.githubusercontent.com/jeyabbalas/py-icare/master/data/BPC3/"

model_covariate_formula_before_cutpoint_url = github_source + "breast_cancer_covariate_model_formula.txt"

model_log_relative_risk_before_cutpoint_url = github_source + "breast_cancer_model_log_odds_ratios.json"
model_log_relative_risk_after_cutpoint_url = github_source + "breast_cancer_model_log_odds_ratios_post_50.json"

model_reference_dataset_before_cutpoint_url = github_source + "reference_covariate_data.csv"
model_reference_dataset_after_cutpoint_url = github_source + "reference_covariate_data_post_50.csv"

apply_covariate_profile_before_cutpoint_url = github_source + "query_covariate_profile.csv"
model_snp_info_url = github_source + "breast_cancer_72_snps_info.csv"
apply_snp_profile_url = github_source + "query_snp_profile.csv"
model_disease_incidence_rates_url = github_source + "age_specific_breast_cancer_incidence_rates.csv"
model_competing_incidence_rates_url = github_source + "age_specific_all_cause_mortality_rates.csv"

In [5]:
# Data will be downloaded here
data_dir = pathlib.Path("data")
data_dir.mkdir(exist_ok=True)

model_covariate_formula_before_cutpoint_path = data_dir / "breast_cancer_covariate_model_formula.txt"

model_log_relative_risk_before_cutpoint_path = data_dir / "breast_cancer_model_log_odds_ratios.json"
model_log_relative_risk_after_cutpoint_path = data_dir / "breast_cancer_model_log_odds_ratios_post_50.json"

model_reference_dataset_before_cutpoint_path = data_dir / "reference_covariate_data.csv"
model_reference_dataset_after_cutpoint_path = data_dir / "reference_covariate_data_post_50.csv"

apply_covariate_profile_before_cutpoint_path = data_dir / "query_covariate_profile.csv"
model_snp_info_path = data_dir / "breast_cancer_72_snps_info.csv"
apply_snp_profile_path = data_dir / "query_snp_profile.csv"
model_disease_incidence_rates_path = data_dir / "age_specific_breast_cancer_incidence_rates.csv"
model_competing_incidence_rates_path = data_dir / "age_specific_all_cause_mortality_rates.csv"

In [6]:
# Download the data
for url, path in zip(
    [
        model_covariate_formula_before_cutpoint_url,
        model_log_relative_risk_before_cutpoint_url,
        model_log_relative_risk_after_cutpoint_url,
        model_reference_dataset_before_cutpoint_url,
        model_reference_dataset_after_cutpoint_url,
        apply_covariate_profile_before_cutpoint_url,
        model_snp_info_url,
        apply_snp_profile_url,
        model_disease_incidence_rates_url,
        model_competing_incidence_rates_url,
    ],
    [
        model_covariate_formula_before_cutpoint_path,
        model_log_relative_risk_before_cutpoint_path,
        model_log_relative_risk_after_cutpoint_path,
        model_reference_dataset_before_cutpoint_path,
        model_reference_dataset_after_cutpoint_path,
        apply_covariate_profile_before_cutpoint_path,
        model_snp_info_path,
        apply_snp_profile_path,
        model_disease_incidence_rates_path,
        model_competing_incidence_rates_path,
    ],
):
    print(f"Downloading {url} to {path}")
    response = requests.get(url)
    response.raise_for_status()
    with open(path, "wb") as f:
        f.write(response.content)

In [7]:
results = icare.compute_absolute_risk_split_interval(
    apply_age_start=30,
    apply_age_interval_length=40,  # 30 + 40 = 70
    cutpoint=50,  # age at which the relative risks and risk factor distributions change in the population
    model_covariate_formula_before_cutpoint_path=model_covariate_formula_before_cutpoint_path,
    model_log_relative_risk_before_cutpoint_path=model_log_relative_risk_before_cutpoint_path,
    model_log_relative_risk_after_cutpoint_path=model_log_relative_risk_after_cutpoint_path,
    model_reference_dataset_before_cutpoint_path=model_reference_dataset_before_cutpoint_path,
    model_reference_dataset_after_cutpoint_path=model_reference_dataset_after_cutpoint_path,
    apply_covariate_profile_before_cutpoint_path=apply_covariate_profile_before_cutpoint_path,
    model_snp_info_path=model_snp_info_path,
    apply_snp_profile_path=apply_snp_profile_path,
    model_family_history_variable_name_before_cutpoint="family_history",
    model_disease_incidence_rates_path=model_disease_incidence_rates_path,
    model_competing_incidence_rates_path=model_competing_incidence_rates_path,
    return_reference_risks=True,  # return the absolute risks for the simulated reference population
    seed=seed  # set the random seed for reproducibility
)

The method returns a dictionary containing the following keys:

In [8]:
results.keys()

dict_keys(['model', 'profile', 'reference_risks', 'method'])

The `method` key contains the name of the iCARE method used:

In [9]:
print(f"iCARE method used: {results['method']}")

iCARE method used: iCARE - absolute risk with split intervals


The `model` key contains the absolute risk model parameters, i.e., the log odds-ratios for each classical risk factor and SNP in association with breast cancer risk. The model parameters are returned as two dictionaries with keys: `before_cutpoint` and `after_cutpoint`. The `before_cutpoint` key contains the model parameters for the age interval before the cut-point, and the `after_cutpoint` key contains the model parameters for the age interval after the cut-point.

In [10]:
model_before_cutpoint = pd.Series(results["model"]["before_cutpoint"])
model_before_cutpoint

C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.11-11.5]    0.044431
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.11.5-12]   -0.035407
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.12-13]     -0.086565
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.13-14]     -0.109902
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.14-15]     -0.085482
                                                                                                             ...   
rs2284378                                                                                                  0.000000
rs2823093                                                                                                 -0.061875
rs17879961                                                              

In [11]:
model_after_cutpoint = pd.Series(results["model"]["after_cutpoint"])
model_after_cutpoint

C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.11-11.5]    0.058212
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.11.5-12]   -0.031099
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.12-13]     -0.089280
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.13-14]     -0.102347
C(age_at_menarche, levels=['<=11', '11-11.5', '11.5-12', '12-13', '13-14', '14-15', '>=15'])[T.14-15]     -0.076935
                                                                                                             ...   
rs2284378                                                                                                  0.000000
rs2823093                                                                                                 -0.061875
rs17879961                                                              

The `profile` key contains the classical risk factors, genetic risk factors, and the estimated absolute risk for each queried profile:

In [12]:
profiles = pd.read_json(results["profile"], orient="records")
profiles.set_index("id", inplace=True)
profiles

,age_interval_start,cutpoint,age_interval_end,age_interval_length,risk_estimates,family_history,age_at_menarche,parity,age_at_first_child_birth,age_at_menopause,...,rs527616,rs1436904,rs6507583,rs4808801,rs3760982,rs2284378,rs2823093,rs17879961,rs132390,rs6001930
id,,,,,,,,,,,,,,,,,,,,,
Q-01,30,50,70,40,0.084664,0,13-14,0,19-22,40-45,...,0,0,0,1,0,1,1,0,0,0
Q-02,30,50,70,40,0.072487,0,>=15,0,19-22,<=40,...,0,0,0,1,1,1,0,0,0,0
Q-03,30,50,70,40,0.144239,0,<=11,0,<=19,51-52,...,1,1,0,1,1,0,0,0,0,0


The `reference_risks` key contains the absolute risks of the reference population. The population estimated risks are stored in the `reference_risks` key. They are returned as two dictionaries with keys: `before_cutpoint` and `after_cutpoint`. The `before_cutpoint` key contains the population estimated risks for the age interval before the cut-point, and the `after_cutpoint` key contains the population estimated risks for the age interval after the cut-point. Each contain a list of dictionaries, one per unique combination of age intervals. Since, we calculated the risks for the interval from age 30 to 70 for all individuals, there is only one dictionary in the list:

In [13]:
pd.DataFrame(results["reference_risks"]["before_cutpoint"][0]["population_risks"]).describe()

,0
count,14137.000000
mean,0.020451
std,0.007284
min,0.006762
25%,0.015299
50%,0.019009
75%,0.024018
max,0.072834


In [14]:
pd.DataFrame(results["reference_risks"]["after_cutpoint"][0]["population_risks"]).describe()

,0
count,14137.000000
mean,0.062535
std,0.022099
min,0.020545
25%,0.046786
50%,0.057957
75%,0.073748
max,0.206095


Note that, unlike the other example notebooks, the example in this notebook specifies two different reference population datasets composed of completely different sets of individuals. So, they cannot be merged into a single distribution. Therefore, in this example, while they helped with calculating the absolute risks, we cannot compare the estimated profile risks with the population risks. We have no combined risk estimates for the whole population.